In [2]:
import pandas as pd
import os
import json

## Read a single JSON

Define a function called read_json. Given a string representing a file path to a json file, this function should open said file and convert its contens into a json object.

In [7]:
# implement me
def read_json(file_path):
    f = open(file_path)
    json_data = json.load(f)
    return json_data

Use the read_json function defined above to read the contents of one of the locations_4.json file acquired in the Data Acquisition Lab.

In [8]:
file_path = os.path.join('./', 'data', 'locations', 'locations_4.json')
json_contents = read_json(file_path)

Display the value of the json_contents variable defined above.

In [9]:
json_contents

{'metadata': {'resultset': {'offset': 1, 'count': 38859, 'limit': 1000}},
 'results': [{'mindate': '1983-01-01',
   'maxdate': '2019-11-16',
   'name': 'Abu Dhabi, AE',
   'datacoverage': 1,
   'id': 'CITY:AE000001'},
  {'mindate': '1974-08-01',
   'maxdate': '2019-11-16',
   'name': 'Mersin, TU',
   'datacoverage': 0.9986,
   'id': 'CITY:TU000042'}]}

## Convert JSON Into a DataFrame

Using the json_contents variable, create a Pandas dataframe.

In [10]:
# Create DataFrame here.
df = pd.json_normalize(json_contents['results'])

Display the contents of the DataFrame. 

In [11]:
# Display contents of the DataFrame here.
df.head()

,mindate,maxdate,name,datacoverage,id
0,1983-01-01,2019-11-16,"Abu Dhabi, AE",1.0000,CITY:AE000001
1,1974-08-01,2019-11-16,"Mersin, TU",0.9986,CITY:TU000042


How many records are in the DataFrame? How many columns does each record have? 

Filter the contents of the DataFrame to only show records where the name column contains the string "Durham".

In [14]:
filtered_df = df[df['name'].str.contains('Durham')]
filtered_df.head()

,mindate,maxdate,name,datacoverage,id


## Read Multiple JSON Files

Define a function called read_all_json_files. Given a string representing a path to a directory, this function should read all of the json files and return a Pandas DataFrame containing all of the objects. In addition to the data from the files, this DataFrame should also contain an additional column called "source". The source column should be populated with the name of the file from which this record originated from.

In [57]:
# implement me
def read_all_json_files(JSON_ROOT):
    df = pd.DataFrame()
    for filename in os.listdir(JSON_ROOT):
        with open(os.path.join(JSON_ROOT, filename), 'r') as file:
            json_data = json.load(file)
            df_2 = pd.json_normalize(json_data['results'])
            df_2['source'] = str(filename)
            df = pd.concat([df,df_2])
    return df

Use the read_all_json_files function above to read the contents of all locations file acquired in the the Data Acquisition Lab.  
Hint: It is easier if you put all of those files in a dedicated subdirectory ex: (./data/locations/)

In [58]:
df = read_all_json_files('./data/locations')

What is the shape of the DataFrame? Does this match the number of columns and records you expected?

In [60]:
# Validate the shape of the DataFrame here
df.shape

(2, 6)

Show the first few records of the DataFrame.

In [66]:
# Show the first few records
df.head()

,mindate,maxdate,name,datacoverage,id,source
0,1983-01-01,2019-11-16,"Abu Dhabi, AE",1.0000,CITY:AE000001,locations_0.json
1,1974-08-01,2019-11-16,"Mersin, TU",0.9986,CITY:TU000042,locations_0.json


Show the last few records of the DataFrame.

In [65]:
# Show the last few records
df.tail()

,mindate,maxdate,name,datacoverage,id,source
0,1983-01-01,2019-11-16,"Abu Dhabi, AE",1.0000,CITY:AE000001,locations_0.json
1,1974-08-01,2019-11-16,"Mersin, TU",0.9986,CITY:TU000042,locations_0.json


## Validate Records Are Unique

Provide two different ways to determine there are no duplicate records.  
Hint: The id field should be unique.

In [67]:
# one day to determine there are not duplicate records
if df['id'].is_unique:
    print("All IDs are unique.")
else:
    print("IDs are not unique.")

All IDs are unique.


In [68]:
# another way to determine there are no duplicate records
if df.duplicated().any():
    print("Duplicate rows exist.")
else:
    print("No duplicate rows.")

No duplicate rows.


# Peking In

Show the record with id "CITY:TU000041". Which file did it come from? 

In [81]:
record = df[df['id'] == "CITY:TU000041"]
if not record.empty:
    print("Record :")
    print(record.iloc[0])
    print("**********")
    print("Source :")
    print(record['source'].iloc[0])

Show the record with id "CLIM:0405". Which file did it come from? 

In [82]:
record = df[df['id'] == "CLIM:0405"]
if not record.empty:
    print("Record :")
    print(record.iloc[0])
    print("**********")
    print("Source :")
    print(record['source'].iloc[0])

How many unique values are there for the source column in the DataFrame?

In [83]:
unique_sources = df['source'].nunique()
print("Number of unique values in the 'source' column:", unique_sources)

Number of unique values in the 'source' column: 1


How many records did the locations_38.json file contain?

In [86]:
records_with_source_38 = df[df['source'] == "locations_38.json"]
num_records_with_source_38 = len(records_with_source_38)
print(num_records_with_source_38)

0


We will work with this DataFrame in a subsequent part of the lab. Instead of repeating all of this work again, let's export the DataFrame to a pickled file.

In [87]:
# export DataFrame to a pickled file called locations_data_frame.pickle. Save it to the data directory.
df.to_pickle("locations_data_frame.pickle")